## Overview

This notebook is intended to:
- Demonstrate how to access time series data for meters and SCADAs using Awesense's Energy Data Model (EDM) SQL API.

Please refer to the [main_concepts.ipynb](../2_main_concepts/main_concepts.ipynb) notebook for a high-level introduction to and simpler examples of the core views and functions available in Awesense's Energy Data Model (EDM).

---

## Set up

In [1]:
import getpass
import plotly.express as px
import plotly.io as pio
import time
import urllib.parse
import pandas as pd

# Allow charts to persist between notebook sessions.
pio.renderers.default='notebook'

**Connection**

Enter the EDM server address and the login credentials provided by Awesense. If you do not have the credentials, or have any trouble connecting, please contact api@awesense.com.
<span style='color:red'> **Please do NOT store the credentials in the notebook, nor share them with anyone.** </span>

In [2]:
edm_address = getpass.getpass(prompt='EDM server address: ')

print('\nEDM login information')
edm_name = getpass.getpass(prompt='Username: ')
edm_password = getpass.getpass(prompt='Password: ')
edm_password = urllib.parse.quote(edm_password)

%load_ext sql
%sql postgresql://$edm_name:$edm_password@$edm_address/edm
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False

# Delete the credential variables for security purpose.
del edm_name, edm_password

EDM server address: ········

EDM login information
Username: ········
Password: ········


---

## Time Series
Various temporary views are created below to optimize query performance within the notebook.

### Meter Time Series

*High level approach*
- Create a temporary view `grid_element_metric`, containing information on meters for grid elements and convenient access to the actual data.
- The `grid_get_downstream()` function is used to gather information of meters that are downstream of a specified grid element.
- Time series are retrieved using the `ts_data_source_select()` function.

In [134]:
#Finding branches in NCZ and only in ABC lines

def branchfind(grid_id = 'North Central Zone', grid_element_id = 'L7-97438'):

    Tree = %sql SELECT grid_element_id, phases, terminal1_cn, terminal2_cn FROM grid_get_downstream('{grid_id}', '{grid_element_id}');
    Tree = Tree.DataFrame()

    #Make sure first element has ABC phases
    if sum(Tree[Tree['grid_element_id']==grid_element_id]['phases'] != 'ABC')==1:
        print(f'{grid_element_id} is not ABC')

    #Find the next link
    AA = Tree[Tree['terminal1_cn']==Tree[Tree['grid_element_id']==grid_element_id]['terminal2_cn'].iloc[0]]
    #If there are at least 2 ABC links connected, then break
    if (sum(AA['phases']=='ABC')>=2):
        print(f"First branching includes {AA[AA['phases']=='ABC']}")
        return AA[AA['phases']=='ABC']

    else:
        # As long as there is only one link
        while (sum(AA['phases']=='ABC')==1):

            #Go on to the next link
            for jj in range(0,len(AA['terminal1_cn'])):
                if AA['phases'].iloc[jj]=='ABC':
                    BB = Tree[(Tree['terminal1_cn']==AA['terminal2_cn'].iloc[jj])]

            AA = BB
    #If there are at least 2 ABC links connected, then break

            if (sum(AA['phases']=='ABC')>=2):
                print(f"First branching includes {AA[AA['phases']=='ABC']}")
                return AA[AA['phases']=='ABC']



In [4]:
def clusfind(grid_id = 'North Central Zone', grid_element_id = 'L7-97438'):

    #Retrieving elements, connections and phases from all elements downstream
    Tree2 = %sql SELECT grid_element_id, terminal1_cn, terminal2_cn, phases FROM grid_get_downstream('{grid_id}', '{grid_element_id}');
    Tree2 = Tree2.DataFrame()

    #Merging elements that come one following the other
    BB = Tree2.merge(Tree2, left_on='terminal2_cn', right_on='terminal1_cn',suffixes=('_upper', '_lower'))

    #Tracing elements where the phases change from ABC to non-ABC
    bif = BB[(BB['phases_upper']=='ABC') & (BB['phases_lower']!='ABC')]
    bif = bif.reset_index()
    return bif


laminar gives the laminar set. IMPORTANT: DO NOT run AA = laminar(...). RUN AA = laminar(...,lami = [],...)
Otherwise, the list will duplicate and the result will get messy. No matter what parameter you use, and even if you use no parameter, lami=[] MUST appear when you call the function. See examples below.

One more note: "Finding branches in NCZ and only in ABC lines" cannot, at this moment, ignore branches that simply leads to a three-phase consumer, such as a factory, when no one\two phases cluster are attached to this branch. This creates somewhat wrong branches and cluster levels downstream from such a consumer

In [114]:
def laminar(grid_id = 'North Central Zone', grid_element_id = 'L7-97438', level = '0', lami = []):
    
    #print(grid_element_id)
    lami_bran = []

    CC = branchfind(grid_id=grid_id, grid_element_id=grid_element_id)
    DD = clusfind(grid_id=grid_id, grid_element_id=grid_element_id)
        
    if len(DD)==0:
        return
    
    lami_clus = DD['grid_element_id_lower'].tolist()
    #print(lami_clus)
    lami_bran.append(lami_clus)
    lami_bran.append(level)
    #print(lami_bran)
    lami.append(lami_bran)
    #print(lami)
    
    ii = 0
    
    try:
        for jj in CC['grid_element_id']:
            level_new = level + str(ii)
            laminar(grid_element_id = jj, level = level_new, lami = lami)
            ii+=1
    except TypeError:
        pass
    
    return lami

In [137]:
lami2 = laminar(lami=[])

First branching includes     grid_element_id phases   terminal1_cn   terminal2_cn
526       L7-100128    ABC  VCN.L7-100134  VCN.L7-100128
691        L7-37052    ABC  VCN.L7-100134   VCN.L7-37052
First branching includes    grid_element_id phases   terminal1_cn   terminal2_cn
10            7462    ABC  VCN.L7-100131       VCN.7462
18       L7-100132    ABC  VCN.L7-100131  VCN.L7-100132


In [138]:
lami2

[[['L7-137713',
   '7831',
   '48331',
   'L7-100133',
   'L7-138029',
   '120341',
   'L7-137915',
   'L7-139409',
   '26033'],
  '0'],
 [['48331', 'L7-100133'], '00'],
 [['L7-100133'], '001'],
 [['L7-137713', '7831', '120341', 'L7-137915', 'L7-139409'], '01']]

In [139]:
lami3 = laminar(grid_element_id='L39-100475', lami=[])

First branching includes      grid_element_id phases    terminal1_cn    terminal2_cn
1079      L39-100485    ABC  VCN.L39-100483  VCN.L39-100485
1187      L39-100665    ABC  VCN.L39-100483  VCN.L39-100665
First branching includes     grid_element_id phases    terminal1_cn    terminal2_cn
135           27099    ABC  VCN.L39-100488       VCN.27099
489      L39-100489    ABC  VCN.L39-100488  VCN.L39-100489


In [129]:
lami3

[[['8AY72ZB',
   'L39-138049',
   '12418',
   'L39-100482',
   '8746',
   '10391',
   'L39-138311',
   'L39-136613',
   'L39-136742',
   '22606',
   '40890',
   '25632',
   'L39-102046',
   '39497',
   '1266',
   '21838',
   '38647',
   'L39-138122',
   'L39-139279',
   '47015',
   'L39-100666',
   'L39-136763',
   '8884',
   'L39-139114',
   '105946',
   'L39-136112',
   '13011',
   'L39-137189',
   'L39-136876',
   '48942',
   '6209',
   '105789',
   '46188',
   '121509',
   'L39-136765',
   '13478',
   'L39-139020',
   '45218',
   'L39-132943',
   'L39-136394',
   'L39-37422',
   'L39-136956',
   'L39-138986',
   '26841'],
  '0'],
 [['10391',
   'L39-138311',
   'L39-136613',
   'L39-136742',
   '22606',
   '40890',
   '25632',
   'L39-102046',
   '39497',
   '1266',
   '21838',
   '38647',
   'L39-138122',
   'L39-139279',
   'L39-37422'],
  '00'],
 [['L39-136613',
   'L39-136742',
   '22606',
   '40890',
   '25632',
   'L39-102046',
   '39497',
   '1266',
   '21838',
   '38647',
 